In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import load_tools
from langchain.memory import ConversationBufferMemory

from rw_formula import RW_Calc2

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
llm = OpenAI(temperature=0)
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:///./rwa_data.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name="risk weight calculator",
        func=RW_Calc2,
        description=
        """calculates a loan risk weight give the argulamts: 
        Segment - Possible values are 'Bank', 'Corporate', and 'Retail'), 
        PD - Probability of Default
        LGD - Loss Given Default
        m - Remaining maturity of the loan in years
        Large_Fin - If 'Y' the client is a Flag for Large Financial Institution, otherwise 'N'
        size - size of the client in MEUR, usually this is the client's turnover
        mortgage - If 'Y' the exposure is a mortgage loan, otherwise 'N'
        revolving - If 'Y' the exposure is a revolving loan, otherwise 'N'
        """
    ),     
    Tool(
        name="rwa_db",
        func=db_chain.run,
        description="useful for quering a database"
    )
]

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    # max_iterations=3,
    memory=memory,
    handle_parsing_errors=True
)

In [ ]:
agent.run("Report the total RWA and EAD per SEGMENT from RWA_122022 table")

In [ ]:
agent.run("What is the sum of all EXPOSURE in RWA_122022 for all ACCOUNT_NUMBER whose STATUS changed from Performing to Non Performing between RWA_122022 table and RWA_032023 table")

In [ ]:
agent.run("Read the information for Client 'sfman997ux8036' in RWA_122022 table and then calculate the RW if PD doubles")